## **ASSIGNMENT 2**

TEAM MEMBERS

*   Sharmadha Parthiban (002100759)
*   Tejal Sadanand Tandel (001563752)

In [ ]:
!pip install demjson
import csv
import dateutil.parser # for parsing plain-text dates
import demjson # json (loose formatting)
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf
from collections import defaultdict
from sklearn import linear_model

     |████████████████████████████████| 131 kB 9.9 MB/s 
  Created wheel for demjson: filename=demjson-2.2.4-py3-none-any.whl size=73565 sha256=a2796d454867cb1f10f7e3ff39bd9ee22df2e1741513d600966679d03bb66455
  Stored in directory: /root/.cache/pip/wheels/41/94/3d/466801f4a8db8e6fce765d7a0115dfebcc55ddf6b00cd98f59
Successfully built demjson




---
# EXCERCISE QUESTIONS: CHAPTER 4

---

# 4.1


---



In [ ]:

path = "/content/fantasy_100.json"
f = open(path)

data = []

for l in f:
    d = json.loads(l)
    data.append(d)
    
f.close()

In [ ]:
data[0]

{'book_id': '18245960',
 'date_added': 'Sun Jul 30 07:44:10 -0700 2017',
 'date_updated': 'Wed Aug 30 00:00:26 -0700 2017',
 'n_comments': 1,
 'n_votes': 28,
 'rating': 5,
 'read_at': 'Sat Aug 26 12:05:52 -0700 2017',
 'review_id': 'dfdbb7b0eb5a7e4c26d59a937e2e5feb',
 'review_text': 'This is a special book. It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind. This is what I love about good science fiction - it pushes your thinking about where things can go. \n It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I\'ve read. For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc. \n It is a book about science, and aliens. The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell. Though when it 

In [ ]:
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
ratingDict = {} # To retrieve a rating for a specific user/item pair

for d in data:
    user,item = d['user_id'], d['book_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = d['rating']

In [ ]:
userAverages = {}
itemAverages = {}

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)

In [ ]:
# function creation
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
# Building a candidate set of items with at least one user in common with the query

def mostSimilar(i, N):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        #sim = Pearson(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:10]

In [ ]:
data[1]

{'book_id': '5577844',
 'date_added': 'Wed Sep 24 09:29:29 -0700 2014',
 'date_updated': 'Wed Oct 01 00:31:56 -0700 2014',
 'n_comments': 1,
 'n_votes': 5,
 'rating': 5,
 'read_at': 'Tue Sep 30 00:00:00 -0700 2014',
 'review_id': '52c8ac49496c153e4a97161e36b2db55',
 'review_text': 'A beautiful story. Neil Gaiman is truly a unique storyteller. I did a combo of reading and listening to this story, and Neil is great at reading as well. I got interested in this book after seeing the movie, which I really enjoyed. The book is similar - maybe a bit more poetic - but I think the plot in the movie is a bit improved (not much, but a bit). \n I imagine this book came about because Neil read the below poem that he includes in the beginning, and then he invented a plot around it. The result is the book reads like a epic poem - it\'s creative, magical, and really, just right. \n Go and catch a falling star, \n Get with child a mandrake root, \n Tell me where all past years are, \n Or who cleft the 

In [ ]:
query = data[1]['book_id']

In [ ]:
ms = mostSimilar(query, 10)
ms

[(1.0, '897461'),
 (1.0, '77566'),
 (1.0, '76620'),
 (1.0, '62291'),
 (1.0, '6'),
 (1.0, '52357'),
 (1.0, '41804'),
 (1.0, '34497'),
 (1.0, '2'),
 (1.0, '18245960')]

In [ ]:
#Faster implementation
def mostSimilarFast(i, N):
    similarities = []
    users = usersPerItem[i]
    candidateItems = set()
    for u in users:
        candidateItems = candidateItems.union(itemsPerUser[u])
    for i2 in candidateItems:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:N]

In [ ]:
#confirming the results
mostSimilarFast(query, 10)

[(1.0, '897461'),
 (1.0, '77566'),
 (1.0, '76620'),
 (1.0, '62291'),
 (1.0, '6'),
 (1.0, '52357'),
 (1.0, '41804'),
 (1.0, '34497'),
 (1.0, '2'),
 (1.0, '18245960')]



---

# 4.2

---



In [ ]:
import math
import random
from collections import defaultdict

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fantasy_100.json to fantasy_100 (1).json


In [ ]:
def simTest(simFunction, nUserSamples):
    sims = []
    randomSims = []

    items = set(usersPerItem.keys())
    users = list(itemsPerUser.keys())

    for u in random.sample(users, nUserSamples):
        itemsU = set(itemsPerUser[u])
        if len(itemsU) < 2: continue # User needs at least two interactions
        (i,j) = random.sample(itemsU, 2)
        k = random.sample(items.difference(itemsU),1)[0]
        usersi = usersPerItem[i].difference(set([u]))
        usersj = usersPerItem[j].difference(set([u]))
        usersk = usersPerItem[k].difference(set([u]))
        sims.append(simFunction(usersi,usersj))
        randomSims.append(simFunction(usersi,usersk))

    print("Average similarity = " + str(sum(sims)/len(sims)))
    print("Average similarity (with random item) = " + str(sum(randomSims)/len(randomSims)))

In [ ]:
simTest(Jaccard, 10)

Average similarity = 0.0
Average similarity (with random item) = 0.125


In [ ]:

def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [ ]:
simTest(CosineSet, 10)

Average similarity = 0.0
Average similarity (with random item) = 0.0




---

# 4.3

---



In [ ]:
items = set(usersPerItem.keys())
users = set(itemsPerUser.keys())

In [ ]:
# 1: Average cosine similarity between i and items in u's history
def rec1score(u, i, userHistory):
    if len(userHistory) == 0:
        return 0
    averageSim = []
    s1 = usersPerItem[i].difference(set([u]))
    for h in userHistory:
        s2 = usersPerItem[h].difference(set([u]))
        averageSim.append(Jaccard(s1,s2))
    averageSim = sum(averageSim)/len(averageSim)
    return averageSim

# 2: Jaccard similarity with most similar user who has consumed i
def rec2score(u, i, userHistory):
    bestSim = None
    for v in usersPerItem[i]:
        if u == v:
            continue
        sim = Jaccard(userHistory, itemsPerUser[v])
        if bestSim == None or sim > bestSim:
            bestSim = sim
    if bestSim == None:
        return 0
    return bestSim

# Generate a recommendation for a user based on a given scoring function
def rec(u, score):
    history = itemsPerUser[u]
    if len(history) > 5: # If the history is too long, just take a sample
        history = random.sample(history,5)
    bestItem = None
    bestScore = None
    for i in items:
        if i in itemsPerUser[u]: continue
        s = score(u, i, history)
        if bestItem == None or s > bestScore:
            bestItem = i
            bestScore = s

    return bestItem, bestScore

In [ ]:
u = random.sample(users,1)[0]

In [ ]:
rec(u, rec1score)

('7304203', 0.0)

In [ ]:
rec(u, rec2score)


('7304203', 0.0)

In [ ]:
def recTest(simFunction, nUserSamples):
    items = set(usersPerItem.keys())
    users = list(itemsPerUser.keys())
    
    better = 0
    worse = 0

    for u in random.sample(users, nUserSamples):
        itemsU = set(itemsPerUser[u])
        if len(itemsU) < 2:
            continue
        i = random.sample(itemsU, 1)[0]
        uWithheld = itemsU.difference(set([i]))
        j = random.sample(items,1)[0]

        si = simFunction(u,i,uWithheld)
        sj = simFunction(u,j,uWithheld)
        
        if si > sj:
            better += 1
        if sj > si:
            worse += 1
            
    print("Better than random " + str(better) + " times")
    print("Worse than random " + str(worse) + " times")


In [ ]:
len(users)

10

In [ ]:
recTest(rec1score,10)

Better than random 0 times
Worse than random 1 times


In [ ]:
recTest(rec2score,10)

Better than random 0 times
Worse than random 0 times




---
#4.4
---



In [ ]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

for d in data:
    user,item = d['user_id'], d['book_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [ ]:
ratingMean = sum([d['rating'] for d in data]) / len(data)
ratingMean

4.09

In [ ]:
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [ ]:
u,i = data[1]['user_id'], data[1]['book_id']
predictRating(u, i)

5.0

In [ ]:
def predictRating1(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    else:
        return ratingMean

In [ ]:
def predictRating2(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerItem[item]:
        u2 = d['user_id']
        if u2 == user: continue
        ratings.append(d['rating'])
        similarities.append(Jaccard(itemsPerUser[user],itemsPerUser[u2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    else:
        return ratingMean

In [ ]:
def predictRating3(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        return ratingMean

In [ ]:
simPredictions1 = [predictRating1(d['user_id'], d['book_id']) for d in data]
simPredictions2 = [predictRating2(d['user_id'], d['book_id']) for d in data]
simPredictions3 = [predictRating3(d['user_id'], d['book_id']) for d in data]

In [ ]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [ ]:
alwaysPredictMean = [ratingMean for d in data]
simPredictions = [predictRating(d['user_id'], d['book_id']) for d in data]
labels = [d['rating'] for d in data]

In [ ]:
MSE(alwaysPredictMean,labels)

1.2819000000000011

In [ ]:
MSE(simPredictions1, labels)

0.8970476034589885

In [ ]:
MSE(simPredictions2, labels)

1.293538000000001

In [ ]:
MSE(simPredictions3, labels)

0.021670415516107856



---

# GOODREADS PROJECT

---



In [ ]:
path = "/content/fantasy_100.json"
f = open(path)

data = []

for l in f:
    d = json.loads(l)
    data.append(d)
    
f.close()

In [ ]:
items = set(usersPerItem.keys())
users = set(itemsPerUser.keys())

In [ ]:
data[0]

{'book_id': '18245960',
 'date_added': 'Sun Jul 30 07:44:10 -0700 2017',
 'date_updated': 'Wed Aug 30 00:00:26 -0700 2017',
 'n_comments': 1,
 'n_votes': 28,
 'rating': 5,
 'read_at': 'Sat Aug 26 12:05:52 -0700 2017',
 'review_id': 'dfdbb7b0eb5a7e4c26d59a937e2e5feb',
 'review_text': 'This is a special book. It started slow for about the first third, then in the middle third it started to get interesting, then the last third blew my mind. This is what I love about good science fiction - it pushes your thinking about where things can go. \n It is a 2015 Hugo winner, and translated from its original Chinese, which made it interesting in just a different way from most things I\'ve read. For instance the intermixing of Chinese revolutionary history - how they kept accusing people of being "reactionaries", etc. \n It is a book about science, and aliens. The science described in the book is impressive - its a book grounded in physics and pretty accurate as far as I could tell. Though when it 

In [ ]:
# 1: Average cosine similarity between i and items in u's history
def rec1score(u, i, userHistory):
    if len(userHistory) == 0:
        return 0
    averageSim = []
    s1 = usersPerItem[i].difference(set([u]))
    for h in userHistory:
        s2 = usersPerItem[h].difference(set([u]))
        averageSim.append(Jaccard(s1,s2))
    averageSim = sum(averageSim)/len(averageSim)
    return averageSim

# 2: Jaccard similarity with most similar user who has consumed i
def rec2score(u, i, userHistory):
    bestSim = None
    for v in usersPerItem[i]:
        if u == v:
            continue
        sim = Jaccard(userHistory, itemsPerUser[v])
        if bestSim == None or sim > bestSim:
            bestSim = sim
    if bestSim == None:
        return 0
    return bestSim

# Generate a recommendation for a user based on a given scoring function
def rec(u, score):
    history = itemsPerUser[u]
    if len(history) > 5: # If the history is too long, just take a sample
        history = random.sample(history,5)
    bestItem = None
    bestScore = None
    for i in items:
        if i in itemsPerUser[u]: continue
        s = score(u, i, history)
        if bestItem == None or s > bestScore:
            bestItem = i
            bestScore = s

    return bestItem, bestScore

In [ ]:
u = random.sample(user,1)[0]

In [ ]:
rec(u, rec1score)

('7304203', 0)

In [ ]:
rec(u, rec2score)

('7304203', 0.0)

In [ ]:
def recTest(simFunction, nUserSamples):
    items = set(usersPerItem.keys())
    users = list(itemsPerUser.keys())
    
    better = 0
    worse = 0

    for u in random.sample(users, nUserSamples):
        itemsU = set(itemsPerUser[u])
        if len(itemsU) < 2:
            continue
        i = random.sample(itemsU, 1)[0]
        uWithheld = itemsU.difference(set([i]))
        j = random.sample(items,1)[0]

        si = simFunction(u,i,uWithheld)
        sj = simFunction(u,j,uWithheld)
        
        if si > sj:
            better += 1
        if sj > si:
            worse += 1
            
    print("Better than random " + str(better) + " times")
    print("Worse than random " + str(worse) + " times")

In [ ]:
recTest(rec1score,10)

Better than random 0 times
Worse than random 0 times


In [ ]:
recTest(rec2score,10)

Better than random 0 times
Worse than random 0 times


# **CONCLUSION**

- Implemented simple item-to-item recommendation strategies.

- Extracted simple features to describe user - item interactions. Builded training set consisting of all pairs (u, i) where books ->i, users ->u.

- Average cosine similarity between i and items in u's history is calculated.

- Jaccard similarity with most similar user who has consumed i is computed.

- Generated recommendation for a user based on a given scoring function